<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/Import_CSV_to_Product_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import CSV to Product Library

Function to import products from a CSV file to Product Library on an entity.
#### Example of CSV format:
```
name,sku,description,price,tax,currency,locale
Bicycle,bic1,This is a nice 2-wheeled bycicle,1200,25,SEK,sv-SE
```

__** NOTE ***__
* Required fields are: **name**, **price**, **currency** and **locale**.
 *  You can set the **price** value to 0 if missing.
* With the **locale** and **currency** options seen below:

label: 'EUR', currencyCode: 'EUR', locale: 'fr-DE'

label: 'Germany - EUR', currencyCode: 'EUR', locale: 'de-DE'

label: 'France - EUR', currencyCode: 'EUR', locale: 'fr-FR'

label: 'Italy - EUR', currencyCode: 'EUR', locale: 'it-IT'

label: 'Spain - EUR', currencyCode: 'EUR', locale: 'es-ES'

label: 'Netherlands - EUR', currencyCode: 'EUR', locale: 'nl-NL'

label: 'Belgium - EUR', currencyCode: 'EUR', locale: 'nl-BE'

label: 'Greece - EUR', currencyCode: 'EUR', locale: 'el-GR'

label: 'Portugal - EUR', currencyCode: 'EUR', locale: 'pt-PT'

label: 'Austria - EUR', currencyCode: 'EUR', locale: 'de-AT'

label: 'Finland - EUR', currencyCode: 'EUR', locale: 'fi-FI'

label: 'Slovakia - EUR', currencyCode: 'EUR', locale: 'sk-SK'

label: 'Ireland - EUR', currencyCode: 'EUR', locale: 'en-IE'

label: 'Lithuania - EUR', currencyCode: 'EUR', locale: 'lt-LT'

label: 'Slovenia - EUR', currencyCode: 'EUR', locale: 'sl-SI'

label: 'Latvia - EUR', currencyCode: 'EUR', locale: 'lv-LV'

label: 'Estonia - EUR', currencyCode: 'EUR', locale: 'et-EE'

label: 'Cyprus - EUR', currencyCode: 'EUR', locale: 'el-CY'

label: 'Luxembourg - EUR', currencyCode: 'EUR', locale: 'lb-LU'

label: 'Malta - EUR', currencyCode: 'EUR', locale: 'mt-MT'

label: 'United States - USD', currencyCode: 'USD', locale: 'en-US'

label: 'United kingdom - GBP', currencyCode: 'GBP', locale: 'en-GB'

label: 'Canada - CAD', currencyCode: 'CAD', locale: 'en-CA'

label: 'Hong kong - HKD', currencyCode: 'HKD', locale: 'en-HK'

label: 'Singapore - SGD', currencyCode: 'SGD', locale: 'en-SG'

label: 'Switzerland - CHF', currencyCode: 'CHF', locale: 'de-CH'

label: 'Japan - JPY', currencyCode: 'JPY', locale: 'ja-JP'

label: 'Sweden - SEK', currencyCode: 'SEK', locale: 'sv-SE'

label: 'Denmark - DKK', currencyCode: 'DKK', locale: 'da-DK'

label: 'Norway - NOK', currencyCode: 'NOK', locale: 'nn-NO'

label: 'Brazil - BRL', currencyCode: 'BRL', locale: 'pt-BR'

label: 'India - INR', currencyCode: 'INR', locale: 'hi-IN'

label: 'South Africa - ZAR', currencyCode: 'ZAR', locale: 'af-ZA'

label: 'Australia - AUD', currencyCode: 'AUD', locale: 'de-AT'

label: 'New Zeeland - NZD', currencyCode: 'NZD', locale: 'en-NZ'

label: 'Thailand - THB', currencyCode: 'THB', locale: 'th-TH'

label: 'Poland - PLN', currencyCode: 'PLN', locale: 'pl-PL'

label: 'Lebanon - LBP', currencyCode: 'LBP', locale: 'en-LB'

label: 'Bahrain - BHD', currencyCode: 'BHD', locale: 'en-BH'

label: 'Jordan - JOD', currencyCode: 'JOD', locale: 'en-JO'

label: 'Saudi Arabia - SAR', currencyCode: 'SAR', locale: 'en-SA'

label: 'United Arab Emirates - AED', currencyCode: 'AED', locale: 'en-AE'

label: 'India - INR', currencyCode: 'INR', locale: 'en-IN'

In [ ]:
# import dependencies
import requests
import os
import io
import json
import copy
import pandas as pd
import ipywidgets as widgets
from google.colab import files

#Constants
BASE_URL = "https://api.getaccept.com/v1"
INT_BASE_URL = "https://int.getaccept.com"
SOURCE_ENTITY_ID = ""
source_auth_headers = {}
#@title ↓↓ Click here to start
#@markdown This step might take a few seconds to run. <br>
#@markdown Then use __shift+enter__ key or click ► left of each step to go through the flow

In [ ]:
email_widget = widgets.Text(
    value="",
    placeholder="Enter login email",
    description="Email:",
    disabled=False
)
password_widget = widgets.Password(
    value="",
    placeholder="Enter password",
    description="Password:",
    disabled=False
)
#@markdown Use the form below to fill in login details to your entity in GetAccept and then run next cell to login <br>
widgets.VBox([email_widget, password_widget])

In [ ]:
#@markdown Login and store API token
if email_widget.value and password_widget.value:
  payload = { "email": email_widget.value, "password": password_widget.value}
  if SOURCE_ENTITY_ID != "":
    payload["entity_id"] = SOURCE_ENTITY_ID
  response = requests.post(BASE_URL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASE_URL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  SOURCE_ENTITY_ID = user_data["user"]["entity_id"]
else:
  raise TypeError("Could not login, missing email or password!")

In [ ]:
#@markdown Select the entity you would like to update the Products to. When you're done, run the next cell
source_entity_list = list(map(lambda x: (x["name"],x["id"]), user_data["entities"]))
source_entity_picker = widgets.Select(
    options=source_entity_list,
    value=SOURCE_ENTITY_ID,
)
source_entity_picker

In [ ]:
#@markdown Verifying entity token of source...
if source_entity_picker.value != SOURCE_ENTITY_ID:
  # Switch entity
  response = requests.get(BASE_URL+"/refresh/"+source_entity_picker.value, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  SOURCE_ENTITY_ID = source_entity_picker.value
print("Token verified")

In [ ]:
#@markdown Upload a CSV file with your products:<br>
uploaded = files.upload()
filePath = "/content/" + list(uploaded.keys())[0]

In [ ]:
#@markdown Import products from CSV
def get_products():
  payload = {"operationName":"getAllProducts","variables":{"searchTerms":"","limit":100,"offset":0,"sort":"desc","sortBy":"createdAt"},"query":"query getAllProducts($searchTerms: String!, $limit: Int!, $offset: Int!, $sort: String!, $sortBy: SortAttributes!) {\n  getAllProducts(searchTerms: $searchTerms, limit: $limit, offset: $offset, sort: $sort, sortBy: $sortBy) {\n    products {\n      ...productListFragment\n      __typename\n    }\n    totalCount\n    __typename\n  }\n}\n\nfragment productListFragment on Product {\n  id\n  sku\n  name\n  price\n  tax\n  description\n  metaData {\n    locale\n    currency\n    __typename\n  }\n  updatedByFullName\n  updatedAt\n  createdByFullName\n  createdAt\n  __typename\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=getAllProducts", json=payload, headers=source_auth_headers)
  product_result = result.json()
  if product_result.get("errors"):
    print(payload)
    print(product_result)
    raise TypeError("Could not get products!")
  else:
    products = product_result["data"]["getAllProducts"]["products"]
    return products

def create_product(prod):
  payload = {"operationName":"CreateProduct","variables":{"payload":{"name":str(prod["name"]), "sku": str(prod["sku"]) if pd.notnull(prod["sku"]) else "","description": str(prod["description"]) if pd.notnull(prod["description"]) else "","price":str(prod["price"]),"tax":str(prod["tax"]) if pd.notnull(prod["tax"]) else "","metaData":{"locale":prod["locale"],"currency":prod["currency"]}}},"query":"mutation CreateProduct($payload: CreateProductInput!) {\n  createProduct(payload: $payload)\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=CreateProduct", json=payload, headers=source_auth_headers)
  create_product_result = result.json()
  if create_product_result.get("errors"):
    print(payload)
    print(create_product_result)
    raise TypeError("Could not create product!")
  else:
    if create_product_result["data"]["createProduct"] == True:
      print("- Product \"" + prod["name"] + "\" added")
    else:
      print("Couldn't add product")

# Get existing products
products = get_products()
# Loop through csv and add products
df = pd.read_csv(filePath)
# Convert column names to lowercase to handle case sensitivity
df.columns = df.columns.str.lower()
for index, row in df.iterrows():
  # Check if product sku exists
  exists = False
#  for prod in products:
#    if prod["sku"] == row["sku"]:
#      exists = True
#      break
  if not exists:
    print("Adding product \"" + row["name"] + "\"")
    create_product(row)
print("Finished importing!")

**Troubleshooting:**

In most cases the CSV file structure is incorrect. You can use this [CSV Product Import Template](https://docs.google.com/spreadsheets/d/1SCGiiAt9k-xsGg6fnyshHft5eR0Q6noc0PzFrSKSRMg/edit?usp=sharing) for inspiration, make a copy and follow the same structure.

The columns must be added in the correct order (name,sku,description,price,tax,currency,locale) and all of the column names are case sensitive. The script will crash if the column **"sku"** is called **"SKU"** or **"Sku"**. Always check the columns for typos if the script crashes.

If the customer shares a CSV file with you, then make sure that the file is comma separated `,` and not separated by semicolon `;`.